In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from time import mktime as mktime
import os
import netCDF4 as nc
from pylab import *
from scipy.optimize import leastsq
from scipy.interpolate import CubicSpline


def get_path(path):
    cwd = str(os.getcwd()).replace('\\', '/')
    path = cwd + path
    return path


# historical load factor data in 2018
wfile = "/Input_data/eco2mix-region-FR-LF.csv"
wfilepath = get_path(wfile)
dw = pd.read_csv(wfilepath, parse_dates=['Date - Heure'])
dw = dw.loc[dw['Date - Heure'].dt.year==2018]

# historical wind farm registre in 2018 
rfilepath = get_path("/Input_data/register.csv")
dr = pd.read_csv(rfilepath, parse_dates=['dateMiseEnService'])
dr['dateMiseEnService'] = dr['dateMiseEnService'].dt.strftime('%d/%m/%y')
dr_np = pd.DataFrame({
    'Latitude': dr['Latitude'],
    'Longitude': dr['Longitude'],
    'Power': dr['puisMaxRac'],
    'Date': dr['dateMiseEnService']
}).to_numpy()

# historical wind speed data import
fn = get_path('/Input_data/ws100.2018.as1e5.FR_025.nc')
f = nc.Dataset(fn) 

# future wind farm registre in 2019
rfilepath = get_path("/Input_data/register.csv")
dr1 = pd.read_csv(rfilepath, parse_dates=['dateMiseEnService'])
dr1['dateMiseEnService'] = dr1['dateMiseEnService'].dt.strftime('%d/%m/%y')
dr1_np = pd.DataFrame({
    'Latitude': dr1['Latitude'],
    'Longitude': dr1['Longitude'],
    'Power': dr1['puisMaxRac'],
    'Date': dr1['dateMiseEnService']
}).to_numpy()

# future wind speed data import
fn1 = get_path('/Input_data/ws100.2019.as1e5.FR_025.nc')
f1 = nc.Dataset(fn1)

# manufacturer power curve
dffilepath = get_path('/Input_data/POWER_CURVE.csv')
df = pd.read_csv(dffilepath)
x = df['WIND SPEED']
y = df['GE 2.5-100']/2500 
y_improved = df['GE 2.5-120']/2500
x.dropna(inplace=True)
y.dropna(inplace=True)
y_improved.dropna(inplace=True)

In [27]:
def geo_idx(dd, dd_array):
    geo_idx = (np.abs(dd_array - dd)).argmin()
    return geo_idx

# wind speed calculation
def regional_representive_ws(f, dr_np):

    lon = f.variables['longitude']
    lat = f.variables['latitude']
    time = f.variables['time']
    ws = f.variables['ws100']
    lons = lon[:]
    lats = lat[:]
    times = time[:]
    t_unit = f.variables['time'].units
    t_cal = f.variables['time'].calendar
    p_times = nc.num2date(times, t_unit, t_cal)  # netCDF wind speed date convert to standard python date format

    # define wind speed start time reference
    ini_date = p_times[0]
    ini_date = mktime(ini_date.timetuple())

    ntimes, ny, nx = shape(ws)
    v_sum = zeros((ntimes), dtype=float)
    v = zeros((ntimes), dtype=float)
    power_sum = zeros((ntimes), dtype=float)

    for v_lat, v_lon, power, date in dr_np:
        date = datetime.datetime.strptime(date, '%d/%m/%y')
        index_date = int((mktime(date.timetuple()) - ini_date) /
                         3600)  # divise 3600 in order to convert to hour

        if index_date <= 0:
            power_list = [power] * ntimes

        elif ntimes < index_date:
            power_list = [0] * ntimes

        else:
            power_list = [0] * index_date
            power_extend = [power] * (ntimes - index_date)
            power_list.extend(power_extend)

        lat_idx = geo_idx(v_lat, lats)
        lon_idx = geo_idx(v_lon, lons)

        v_sum += power_list[:] * np.power(ws[:, lat_idx, lon_idx].data, 3)
        power_sum = power_sum + power_list

    v = np.power(v_sum / power_sum, 1 / 3)

    ws_ts = pd.DataFrame({
        'Date': p_times,
        'Wind speed': v,
    })

    ws_ts['Date'] = pd.to_datetime(ws_ts['Date'], unit='h')

    return ws_ts


def fun_y_fit(x, p):
    a, c, k = p
    y_fit = a * np.arcsin(((1 - np.exp(-(x / c)**k))))
    return y_fit


def residuals(p, y, x):
    return y - fun_y_fit(x, p)


def power_curve_parameter(ws_ts, dw):
    # LF and WS reading
    x = ws_ts['Wind speed']
    y = dw['LF Wind']

    # sorted x,y
    x0 = np.sort(x)
    y0 = np.sort(y)

    y_epc = x.apply(
        lambda a: y0[int(len(y0) * (np.abs(x0 - a)).argmin() / len(x0))
                     ])  # empirical power curve load factor

    # fitting empirical power curve
    p0 = [0.1, 0.3, 0.5]
    plsq = leastsq(residuals, p0, args=(y_epc, x))

    return plsq[0]


def load_factor_calculation(coeff, ws_ts):
    x = ws_ts['Wind speed']
    y_pred = fun_y_fit(x, coeff)

    lf_ts = pd.DataFrame({
        'Date': ws_ts['Date'],
        'LF': y_pred,
    })

    return lf_ts


def fun_y_fit1(x, p):
    a, c, k = p
    y_fit = 2 / 3.1415926 * np.arcsin(((1 - np.exp(-(x / c)**k))))
    return y_fit


def residuals1(p, y, x):
    return y - fun_y_fit1(x, p)


def improved_coeff(coeff, cs, cs_improved, x_range):
    
    p0 = [2 / 3.1415926, 0.3, 0.5]
    plsq = leastsq(residuals1, p0, args=(cs(x_range), x_range))
    plsq_improved = leastsq(residuals1, p0, args=(cs_improved(x_range), x_range))
    coeff_improved = plsq_improved[0] * coeff / plsq[0]
    
    return coeff_improved
    
def improved_ws (h1, h2, z0, ws_ts):
    x = ws_ts['Wind speed']
    alpha = (z0/h1)**0.2*(1-0.55*np.log10(x))
    improved_ws = x * (h2/h1)**alpha
    
    improved_ws = pd.DataFrame({
        'Date': ws_ts['Date'],
        'Wind speed': improved_ws,
    })
    
    return improved_ws
    

def data_save(df, path):
    fullname = os.path.join('/Output_data/', path)
    df.to_csv(get_path(fullname), index=False)
    
    

In [5]:
# calculate historical regional representative wind speed time series
ws_ts = regional_representive_ws(f, dr_np)

In [7]:
# calcualte power curve coeff using Empirical Parametric CDF model
coeff = power_curve_parameter(ws_ts, dw)

/homel/ylcai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in arcsin


In [6]:
# calculate future regional representative wind speed time series
ws_ts1 = regional_representive_ws(f1, dr1_np)

In [8]:
# predict future regional load factor time series
lf_pred = load_factor_calculation(coeff, ws_ts1)

In [28]:
# improved power curve coeff
cs = CubicSpline(x, y)
cs_improved = CubicSpline(x, y_improved)
x_range = np.arange(0, 25, 0.01)
coeff_improved = improved_coeff(coeff, cs, cs_improved, x_range)

# improved wind speed
h1 = 80
h2 = 120
z0 = 0.03
ws_ts1_improved = improved_ws (h1, h2, z0, ws_ts1)

# predict improved future regional load factor time series
lf_pred_improved = load_factor_calculation(coeff_improved, ws_ts1_improved)

In [32]:
# data save
ws_ts_outpath = 'eco2mix-region-FR-historical-WS.csv'
coeff_outpath = 'power_curve_coeff.csv'
coeff_improved_outpath = 'power_curve_improved_coeff.csv'
ws_ts1_outpath = 'eco2mix-region-FR-future-WS.csv'
lf_pred_outpath = 'eco2mix-region-FR-future-LF.csv'
lf_pred_improved_outpath = 'eco2mix-region-FR-future-improved-LF.csv'

data_save(ws_ts, ws_ts_outpath)
coeff1 = pd.DataFrame(coeff)
data_save(coeff1, coeff_outpath)
coeff1_improved = pd.DataFrame(coeff_improved)
data_save(coeff1_improved, coeff_improved_outpath)
data_save(ws_ts1, ws_ts1_outpath)
data_save(lf_pred, lf_pred_outpath)
data_save(lf_pred_improved, lf_pred_improved_outpath)